In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, ArrayType, IntegerType
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import MinHashLSH

import random

config = SparkConf().setAppName("LSH").setMaster("local[8]")
sc = SparkContext.getOrCreate(config)
spark = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/20 09:19:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/20 09:19:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/20 09:19:44 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/04/20 09:19:44 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/04/20 09:19:44 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
23/04/20 09:19:44 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.


# RDD 形式

In [31]:
import re

SHINGLE_LENGTH = 3
PRIME = 2147483647
SIMILARITY_THRESHOLD = 0.001
BAND_SIMILARITY_THRESHOLD = 0.001
NUMBER_OF_BANDS = 10
NUMBER_OF_ROWS = 5
NUMBER_OF_HASHFUNCTIONS = 10


def preprocessDocument(document):
    return document.strip().lower().replace("[^\\w\\s]", "").replace("\\s+", " ")

def shingle(document):
  resultingList = []
  i = 0
  while i + SHINGLE_LENGTH < len(document):
    resultingList.append(hash(document[i:i+SHINGLE_LENGTH]))
    i += 1
  return resultingList

def minHash(listOfShingles, hashFunctions):
  result = []
  for ind in range(len(hashFunctions)):
    minVal = float("inf")
    for shingle in listOfShingles:
      hashResult = hashFunctions[ind](shingle)
      if hashResult < minVal:
        minVal = hashResult
    result.append(minVal)
  return result

def getHashFunctions(number):
  result = []
  rand = random.Random()
  for i in range(number):
    a = rand.randint(0, PRIME-1)
    b = rand.randint(0, PRIME-1)
    result.append(lambda x, a=a, b=b: ((a * x + b) % PRIME))
  return result

def signatureToHashedBandsOfRows(signature, numberOfBands, numberOfRowsInBand):
  if len(signature) != numberOfBands * numberOfRowsInBand:
    raise Exception("Wrong arguments number of bands times number of rows should equal length of signature")
  i = 0
  bands = []
  while i + numberOfRowsInBand <= len(signature):
    bands.append(signature[i:i+numberOfRowsInBand])
    i += numberOfRowsInBand
  return [hash(tuple(band)) for band in bands]

def findSimilarity(ints, ints1):
  intersection = set(ints).intersection(set(ints1))
  union = set(ints).union(set(ints1))
  return len(intersection) / len(union)

In [32]:

from pyspark.sql.functions import udf,col

# sharedHashFunctions = sc.broadcast(getHashFunctions(NUMBER_OF_HASHFUNCTIONS))
sharedHashFunctions = getHashFunctions(NUMBER_OF_HASHFUNCTIONS)

textFile = sc.textFile("../dataset/text.txt")
documents = textFile.zipWithIndex().map(lambda x: (x[1], preprocessDocument(x[0])))
shingled = documents.map(lambda x: (x[0], shingle(x[1])))
minHashed = shingled.map(lambda x: (x[0], minHash(x[1], sharedHashFunctions)))

similarities = minHashed.cartesian(minHashed) \
  .filter(lambda x: x[0][0] > x[1][0]) \
  .map(lambda x: (x[0][0], x[1][0], findSimilarity(x[0][1], x[1][1]))) \
  .filter(lambda x: x[2] > SIMILARITY_THRESHOLD)
similarities.toDF().show()
# print(similarities.collect())

bands = minHashed.map(lambda x: (x[0], signatureToHashedBandsOfRows(x[1], NUMBER_OF_BANDS, NUMBER_OF_ROWS)))
bands.cache()
similarities.toDF().show()


# lsh = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=NUMBER_OF_BANDS)

# data = [(Vectors.dense(band), [i]) for (i, band) in bands.collect()]
# df = spark.createDataFrame(data, ["features", "id"])
# model = lsh.fit(df)
# model.transform(df).show()

# result = model.approxSimilarityJoin(df, df, BAND_SIMILARITY_THRESHOLD, distCol="JaccardDistance") 
# result = result.withColumn("Similariy", 1-col("JaccardDistance"))
#   # .filter("JaccardDistance > " + str(BAND_SIMILARITY_THRESHOLD)) \
  

# result.select("datasetA.id", "datasetB.id", "JaccardDistance").show(5)

+---+---+-------------------+
| _1| _2|                 _3|
+---+---+-------------------+
|  1|  0| 0.8181818181818182|
|  3|  0|0.05263157894736842|
|  3|  1|0.05263157894736842|
|  3|  2|0.05263157894736842|
+---+---+-------------------+

+---+---+-------------------+
| _1| _2|                 _3|
+---+---+-------------------+
|  1|  0| 0.8181818181818182|
|  3|  0|0.05263157894736842|
|  3|  1|0.05263157894736842|
|  3|  2|0.05263157894736842|
+---+---+-------------------+



# Dataframe 形式

In [100]:
import re

SHINGLE_LENGTH = 3
PRIME = 2147483647
SIMILARITY_THRESHOLD = 0.001
BAND_SIMILARITY_THRESHOLD = 0.001
NUMBER_OF_BANDS = 10
NUMBER_OF_ROWS = 5
NUMBER_OF_HASHFUNCTIONS = 20


def preprocessDocument(document):
    return document.strip().lower().replace("[^\\w\\s]", "").replace("\\s+", " ") if isinstance(document, str) else document

def shingle(document):
  resultingList = []
  i = 0
  while i + SHINGLE_LENGTH < len(document):
    resultingList.append(hash(document[i:i+SHINGLE_LENGTH]))
    i += 1
  return resultingList

def minHash(listOfShingles, hashFunctions):
  result = []
  for ind in range(len(hashFunctions)):
    minVal = float("inf")
    for shingle in listOfShingles:
      hashResult = hashFunctions[ind](shingle)
      if hashResult < minVal:
        minVal = hashResult
    result.append(minVal)
  return result

def getHashFunctions(number):
  result = []
  rand = random.Random()
  for i in range(number):
    a = rand.randint(0, PRIME-1)
    b = rand.randint(0, PRIME-1)
    result.append(lambda x, a=a, b=b: ((a * x + b) % PRIME))
  return result

def signatureToHashedBandsOfRows(signature, numberOfBands, numberOfRowsInBand):
  if len(signature) != numberOfBands * numberOfRowsInBand:
    raise Exception("Wrong arguments number of bands times number of rows should equal length of signature")
  i = 0
  bands = []
  while i + numberOfRowsInBand <= len(signature):
    bands.append(signature[i:i+numberOfRowsInBand])
    i += numberOfRowsInBand
  return [hash(tuple(band)) for band in bands]

def findSimilarity(ints, ints1):
  print(ints, ints1)
  intersection = set(ints).intersection(set(ints1))
  union = set(ints).union(set(ints1))
  return len(intersection) / len(union)

In [101]:
from pyspark.sql.functions import udf,col
from pyspark.sql.types import LongType, FloatType

# sharedHashFunctions = sc.broadcast(getHashFunctions(NUMBER_OF_HASHFUNCTIONS))
sharedHashFunctions = getHashFunctions(NUMBER_OF_HASHFUNCTIONS)

text_df = spark.read.text("../dataset/text.txt").withColumnRenamed("value", "text")

# 自定义 UDF，为每行分配一个连续的整数 ID
counter = iter(range(text_df.count()))
row_id_udf = udf(lambda _: next(counter), LongType())

# 为 DataFrame 增加行号
text_df = text_df.withColumn("id", row_id_udf("text"))

# 处理文本
preprocessDocument_udf = udf(preprocessDocument, StringType())
text_df = text_df.withColumn("text", preprocessDocument_udf("text"))

# 分词
shingle_udf = udf(shingle, ArrayType(LongType()))
text_df = text_df.withColumn("shingles", shingle_udf("text"))

# MinHash
minHash_udf = udf(lambda x: minHash(x, sharedHashFunctions), ArrayType(LongType()))
text_df = text_df.withColumn("minHash", minHash_udf("shingles"))
text_df.show()

# 相似度 udf
jaccard_similarity = udf(lambda x, y: len(
            set(x).intersection(set(y))) / len(set(x).union(set(y))), FloatType())

# 相似度
similarities = text_df.alias("left").crossJoin(text_df.alias("right")) \
    .filter(col("right.id") > col("left.id"))  \
    .withColumn("similarity", jaccard_similarity(col("left.minHash"), col("right.minHash")))  \
    # .filter(col("similarity") > SIMILARITY_THRESHOLD)

similarities.select("left.id", "right.id", "similarity").show()

# bands = minHashed.map(lambda x: (x[0], signatureToHashedBandsOfRows(x[1], NUMBER_OF_BANDS, NUMBER_OF_ROWS)))
# bands.cache()
# similarities.toDF().show()

+--------------------+---+--------------------+--------------------+
|                text| id|            shingles|             minHash|
+--------------------+---+--------------------+--------------------+
|you could create ...|  0|[-451277881344813...|[27707836, 331706...|
|you could do a da...|  1|[-451277881344813...|[27707836, 331706...|
|the document simi...|  2|[6618406970764183...|[141778275, 33170...|
|  the lazy black cat|  3|[6618406970764183...|[15180064, 164757...|
+--------------------+---+--------------------+--------------------+

+---+---+-----------+
| id| id| similarity|
+---+---+-----------+
|  0|  1| 0.73913044|
|  0|  2| 0.05263158|
|  0|  3|0.025641026|
|  1|  2| 0.08108108|
|  1|  3|0.025641026|
|  2|  3|        0.0|
+---+---+-----------+



# Dataframe 封装成 class

In [135]:
import re

SHINGLE_LENGTH = 3
PRIME = 2147483647
SIMILARITY_THRESHOLD = 0.001
BAND_SIMILARITY_THRESHOLD = 0.001
NUMBER_OF_BANDS = 10
NUMBER_OF_ROWS = 5
NUMBER_OF_HASHFUNCTIONS = 20

class minLSH:
    def __init__(self, numHashTables=5, shingleSize=5, inputCol="features", outputCol="hashes"):
        self.numHashTables = numHashTables
        self.shingleSize = shingleSize
        self.inputCol = inputCol
        self.outputCol = outputCol
        self.hashFunctions = self.getHashFunctions(self.numHashTables)

    def getHashFunctions(self, number):
        result = []
        rand = random.Random()
        for _ in range(number):
            a = rand.randint(0, PRIME-1)
            b = rand.randint(0, PRIME-1)
            result.append(lambda x, a=a, b=b: ((a * x + b) % PRIME))
        return result
    
    def preprocessDocument(self, document):
        return document.strip().lower().replace("[^\\w\\s]", "").replace("\\s+", " ") if isinstance(document, str) else document

    def shingle(self, document):
        resultingList = []
        i = 0
        while i + self.shingleSize < len(document):
            resultingList.append(hash(document[i:i+self.shingleSize]))
            i += 1
        return resultingList

    def minHash(self, listOfShingles):
        result = []
        for ind in range(len(self.hashFunctions)):
            minVal = float("inf")
            for shingle in listOfShingles:
                hashResult = self.hashFunctions[ind](shingle)
                if hashResult < minVal:
                    minVal = hashResult
            result.append(minVal)
        return result
    
    def signatureToHashedBandsOfRows(self, signature, numberOfBands, numberOfRowsInBand):
        if len(signature) != numberOfBands * numberOfRowsInBand:
            raise Exception("Wrong arguments number of bands times number of rows should equal length of signature")
        i = 0
        bands = []
        while i + numberOfRowsInBand <= len(signature):
            bands.append(signature[i:i+numberOfRowsInBand])
            i += numberOfRowsInBand
        return [hash(tuple(band)) for band in bands]

    def fit(self, dataframe):
        return self

    def transform(self, dataframe):
        preprocessDocument_udf = udf(self.preprocessDocument, StringType())
        df = dataframe.withColumn("document", preprocessDocument_udf(self.inputCol))
        # 分词
        shingle_udf = udf(self.shingle, ArrayType(LongType()))
        df = df.withColumn("shingles", shingle_udf("document")).drop("document")
        # MinHash
        minHash_udf = udf(lambda x: self.minHash(x), ArrayType(LongType()))
        df = df.withColumn("minHash", minHash_udf("shingles")).drop("shingles")
        return df

    def approxSimilarityJoin(self, dataframeA, dataframeB, threshold):
        # 相似度 udf
        jaccard_similarity = udf(lambda x, y: len(
                    set(x).intersection(set(y))) / len(set(x).union(set(y))), FloatType())

        # 相似度
        similarities = dataframeA.alias("left").crossJoin(dataframeB.alias("right")) \
            .withColumn("similarity", jaccard_similarity(col("left.minHash"), col("right.minHash")))  \
            .filter(col("similarity") >= threshold)

        return similarities



In [137]:
text_df = spark.read.text("../dataset/text.txt").withColumnRenamed("value", "text")

# 自定义 UDF，为每行分配一个连续的整数 ID
counter = iter(range(text_df.count()))
row_id_udf = udf(lambda _: next(counter), LongType())

# 为 DataFrame 增加行号
text_df = text_df.withColumn("id", row_id_udf("text"))
model = minLSH(numHashTables=10, shingleSize=3, inputCol="text", outputCol="hashes")
model = model.fit(text_df)
text_df = model.transform(text_df)
text_df.show()
result = model.approxSimilarityJoin(text_df, text_df, 0)
result.where("left.id>right.id").select("left.id", "right.id", "similarity").show()


+--------------------+---+--------------------+
|                text| id|             minHash|
+--------------------+---+--------------------+
|you could create ...|  0|[57937548, 580122...|
|you could do a da...|  1|[57937548, 580122...|
|the document simi...|  2|[30349375, 778723...|
|  The lazy black cat|  3|[23535873, 778723...|
+--------------------+---+--------------------+

+---+---+----------+
| id| id|similarity|
+---+---+----------+
|  1|  0| 0.8181818|
|  2|  0|       0.0|
|  2|  1|       0.0|
|  3|  0|       0.0|
|  3|  1|       0.0|
|  3|  2|0.05263158|
+---+---+----------+

